In [1]:
import requests, bs4
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [115]:
#gather 2019 data
ext = ["", "/101", '/201', '/301', '/401', '/501']
base_url = 'https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/released-in-2019'
page_dict= []
link_list = []
for i in ext:
    url = str(base_url+i)
    response = requests.get(url)
    page= response.text
    soup = bs(page)
    #parsing table data into two lists
    keys = soup.find('thead').find_all('th')
    keys_list = [item.text for item in keys]

    values = soup.find('tbody').find_all('tr')
    rows_list = [(item.text.strip("\n")
              .rstrip('\n').replace('$', '')
              .replace(",", "").replace('\n', ',')) for item in values]

    #reformat each row string into dictionary
    for i in rows_list:
        i = i.split(",")
        for b in i[:2]:
            b= str(b)
        movie_info = dict(zip(keys_list, i))
        page_dict.append(movie_info)



In [259]:
#gather top grossing 2017- 2019 data
ext = ["", "/101", '/201', '/301', '/401', '/501']
year = ["released-in-2017", "released-in-2018", "released-in-2019"]
base_url = 'https://www.the-numbers.com/box-office-records/domestic/all-movies/cumulative/'
page_dict= []
all_links = []
for i in year:
    url = str(base_url+i)
    for i in ext:
        full_url = str(url+i)
        response = requests.get(full_url)
        page= response.text
        soup = bs(page)
        #parsing table data into key, value lists. getting hrefs.
        
        links = soup.find('tbody').find_all('a')
        link_list = [item['href'] for item in links]
        link_list=link_list[::2]
        all_links.append(link_list)
        
        keys = soup.find('thead').find_all('th')
        keys_list = [item.text for item in keys]


        values = soup.find('tbody').find_all('tr')
        rows_list = [(item.text.strip("\n")
                      .rstrip('\n').replace('$', '')
                      .replace(",", "").replace('\n', ',')
                         .replace("â\\x80\\x99ll", "'")
                         .split(",")) for item in values]
       

        #reformat each row string into dictionary
        for i in rows_list:
            movie_info = dict(zip(keys_list, i))
            page_dict.append(movie_info)

topline_info = pd.DataFrame(page_dict)
topline_info

In [495]:
topline_info.head()

,Rank,Movie,Distributor,DomesticBoxOffice,OpeningWeekendBox Office,MaxTheaterCount
0,1,Star Wars Ep. VIII: The Last Jedi,Walt Disney,620181382,220009584,4232
1,2,Beauty and the Beast,Walt Disney,504014165,174750616,4210
2,3,Wonder Woman,Warner Bros.,412563408,103251471,4165
3,4,Jumanji: Welcome to the Jungle,Sony Pict…,404508916,36169328,3849
4,5,Guardians of the Galaxy Vol 2,Walt Disney,389813101,146510104,4347


In [515]:
#add all the movie-specific page links to a list
movie_links  = []
for i in all_links:
    for c in i:
        movie_links.append(c)

In [516]:
print(movie_links[:20])

['/movie/Star-Wars-Ep-VIII-The-Last-Jedi#tab=summary', '/movie/Beauty-and-the-Beast-(2017)#tab=summary', '/movie/Wonder-Woman-(2017)#tab=summary', '/movie/Jumanji-Welcome-to-the-Jungle#tab=summary', '/movie/Guardians-of-the-Galaxy-Vol-2#tab=summary', '/movie/Spider-Man-Homecoming#tab=summary', '/movie/It-(2017)#tab=summary', '/movie/Thor-Ragnarok#tab=summary', '/movie/Despicable-Me-3#tab=summary', '/movie/Justice-League-(2017)#tab=summary', '/movie/Logan-(2017)#tab=summary', '/movie/Fate-of-the-Furious-The#tab=summary', '/movie/Coco-(2017)#tab=summary', '/movie/Dunkirk-(2017)#tab=summary', '/movie/Get-Out-(2017)#tab=summary', '/movie/Lego-Batman-Movie-The#tab=summary', '/movie/Boss-Baby-The#tab=summary', '/movie/Greatest-Showman-The#tab=summary', '/movie/Pirates-of-the-Caribbean-Dead-Men-Tell-No-Tales#tab=summary', '/movie/Kong-Skull-Island#tab=summary']


## Gather movie-specific data into a list of dictionaries

In [601]:
def get_title(soup):
    movie = soup.find("h1").text
    n = len(movie)
    movie = movie[:n-6].rstrip(" ").replace("\x80\x99s", "").replace("â", "'s")
    return movie

In [602]:
from collections import ChainMap
movie_details = []
base_url = 'https://the-numbers.com'
for i in movie_links:
    url = base_url + i
    response = requests.get(url)
    page= response.text
    soup = bs(page)

#gathering crew

    crew_dict ={}
    try:
        production_list = [item.text.strip('\n')
                       .rstrip('\n').replace("\n\xa0\xa0\n","=")
                       for item in soup.find(
                           text=re.compile("Production and Technical Credits")).next.next.find_all("tr")]  
    except:
        pass

    try:           
        production_list = [s.split("=") for s in production_list]
        production_dict = dict([(b, a) for [a,b] in production_list])
    except:
        pass
    
    try: 
        crew_dict["Director"] = production_dict["Director"]
    except:
        crew_dict["Director"] = None
        
    try:   
        crew_dict["Producer"] = production_dict["Producer"]
    except:
        crew_dict["Producer"] =None
    try:
        crew_dict['Screenwriter'] = production_dict['Screenwriter']
    except:
        crew_dict['Screenwriter'] = None
        

    #add only director producer screenwriter

#gathering cast

    cast_dict= {}
    try:
        cast_list = [item.text.strip('\n')
                 .rstrip('\n').replace("\n\xa0\xa0\n","=")
                 for item in soup.find(class_="cast_new").next.next.parent.find_all("tr")]
    except:
        pass
    
    try:
        cast_list = [s.split("=")[0] for s in cast_list]
    except:
        pass
    
    try:
        cast_dict["lead_1"] = cast_list[0]
    except:
        cast_dict["lead_1"] = None
    try: 
        cast_dict["lead_2"] = cast_list[1]
    except:
        cast_dict["lead_2"] = None
    try:
        cast_dict["lead_3"] = cast_list[2]
    except:
        cast_dict["lead_3"] = None
    try:
        cast_dict["lead_4"] = cast_list[3]
    except:
        cast_dict["lead_4"] = None
        
#put into list with 

#metrics
    try:
        metrics_list = [item.text.strip('\n').rstrip('\n').replace("\xa0", " ")
                    .replace("$", ""). replace(",", "")
                    .replace("Opening Weekend", "Op_Wknd_Total_Gross_Share").replace('\n', ':')
                    for item in soup.find(id="movie_ratings").next.next.next.next.find_all("tr")]
    except:
        pass
    
    try:
        metrics_dict = dict([s.split(":") for s in metrics_list])
    except:
        metrics_dict = {}

#reformating Dictionary 

    try:
        metrics_dict['Op_Wknd_Total_Gross_Share'] = float((metrics_dict[
        'Op_Wknd_Total_Gross_Share'].split(" ")[1].strip("(").rstrip('%')))
    except: 
        metrics_dict['Op_Wknd_Total_Gross_Share'] = None
    try:
        metrics_dict["Legs"] = float(metrics_dict["Legs"].split(" ")[0])
    except:
        metrics_dict["Legs"] = None
    try:
        metrics_dict['Domestic Share'] = float(metrics_dict['Domestic Share'].split(" ")[0].rstrip('%'))
    except:
        metrics_dict['Domestic Share'] = None
    try:
        metrics_dict["Production Budget"] = float(metrics_dict["Production Budget"].split(" ")[5])
    except:
        metrics_dict["Production Budget"] = None
    try:
        metrics_dict["average_run_wks"] = float(metrics_dict["Theater counts"].split(" ")[5])
    except: 
        metrics_dict["average_run_wks"] = None
    try:
        metrics_dict["Theater counts"] = int(metrics_dict["Theater counts"].split(" ")[0])
    except:
        metrics_dict["Theater counts"] = None
        
    try:
        metrics_dict["Infl. Adj. Dom. BO"] = int(metrics_dict["Infl. Adj. Dom. BO"])
    except:
         metrics_dict["Infl. Adj. Dom. BO"] = None

#delete repeat columns

    try:
        del metrics_dict["Theater counts"]
    except:
        pass
    #gathering details
    try:
        details_list = [item.text.rstrip("\n").replace('\n', "").replace("\xa0", "_") for item in 
                soup.find(text= re.compile("Movie Details")).next.next.find_all("tr")]
    except:
        pass

#delete extra columns:
    try: 
        details_list = [s.split(":") for s in details_list]
    except:
        pass
    try:
        details_dict = dict([(x[0], x[1:]) for x in details_list])
    except:
        details_dict = {"Domestic Release": None, 'MPAA_Rating': None, "Running Time": None,
                       "Franchise": None, "Source": None, "Original Screenplay": None,
                       "Genre":None, }
    try:
        del details_dict["Languages"]
    except:
        pass
    try:
        del details_dict['International Releases']
    except:
        pass
    try:
        del details_dict['Video_Release']
    except:
        pass
    try:
        del details_dict['Comparisons']
    
    except:
        pass
    try:
        del details_dict['Keywords']
    except:
        pass

#reformat values:

    try:
        details_dict['Domestic Releases'] =[i.split(" ") for i in details_dict['Domestic Releases']][0][:3]
    except: 
        pass
    try:
        details_dict['Domestic Releases'] = ("{}"+" "+"{}"+" "+"{}").format(
            details_dict['Domestic Releases'][0],
            details_dict['Domestic Releases'][1], 
            details_dict['Domestic Releases'][2])
    except:
        details['Domestic Releases'] = None

    try:
        details_dict['MPAA_Rating'] = [i.split(" ") for i in details_dict['MPAA_Rating']][0][0]
    except:
        details_dict['MPAA_Rating'] = None
        
    try:
        details_dict['Running Time'] = [i.split(" ") for i in details_dict['Running Time']][0][0]
    except: 
        details_dict['Running Time'] = None
    try:
        details_dict['Franchise'] = details_dict['Franchise'][0]
    except:
        details_dict['Franchise'] = None
    try:
        details_dict['Source'] = details_dict['Source'][0]
    except:
        details_dict['Source'] = None
        
    try:
        details_dict['Genre'] = details_dict['Genre'][0]
    except:
        details_dict['Genre'] = None 
    try:
        details_dict['Production_Method'] = details_dict['Production_Method'][0]
    except: 
        details_dict['Production_Method'] = None
    try:
        details_dict['Creative_Type'] = details_dict['Creative_Type'][0]
    except:
        details_dict['Creative_Type'] = None

    
    
    movie = get_title(soup)

    details_dict["movie"] = movie
    
    movie_dict = dict(ChainMap(details_dict, metrics_dict, cast_dict, production_dict))
    movie_details.append(movie_dict)

In [605]:
movie_df = pd.DataFrame(movie_details)

In [613]:
movie_df["Legs"]

0       2.82
1       2.88
2       4.00
3       8.08
4       2.66
        ... 
1744    1.00
1745    1.00
1746    1.00
1747    1.00
1748    1.00
Name: Legs, Length: 1749, dtype: float64

In [612]:
topline_info["Movie"]

0       Star Wars Ep. VIII: The Last Jedi
1                    Beauty and the Beast
2                            Wonder Woman
3          Jumanji: Welcome to the Jungle
4           Guardians of the Galaxy Vol 2
                      ...                
1744                              Pupille
1745                               Xiao Q
1746                    The Killing Floor
1747                  The Hours and Times
1748                A Fish in the Bathtub
Name: Movie, Length: 1749, dtype: object

In [614]:
movies_df = pd.merge(topline_info, movie_df, left_on ="Movie", right_on='movie')

In [616]:
movies_df.describe

<bound method NDFrame.describe of      Rank                              Movie   Distributor DomesticBoxOffice  \
0       1  Star Wars Ep. VIII: The Last Jedi   Walt Disney         620181382   
1       2               Beauty and the Beast   Walt Disney         504014165   
2       3                       Wonder Woman  Warner Bros.         412563408   
3       4     Jumanji: Welcome to the Jungle    Sony Pict…         404508916   
4       5      Guardians of the Galaxy Vol 2   Walt Disney         389813101   
...   ...                                ...           ...               ...   
1623  553                            Pupille    Distrib F…              1737   
1624  554                             Xiao Q   Well Go USA              1652   
1625  555                  The Killing Floor    Film Move…              1583   
1626  556                The Hours and Times    Oscillosc…              1273   
1627  557              A Fish in the Bathtub    Cohen Med…              1237   

     

In [619]:
import pickle 
f = open("merged_data_raw", "wb")
pickle.dump("movies_df", f)
f.close()



In [624]:
movies_df.to_csv("movies_raw.csv")